<h1>Final Project Tutorial - Taeyoon Kim, Victor Chang</h1>

In this tutorial, you will apply your data science skills to see how well we can classify a mushroom by it's edibility.

<h2>Introduction</h2>
<p>In this tutorial, you will apply your data science skills to see how well we can classify a mushroom by it's edibility. We will be using mushroom data provided by UC Irvine to train our machine learning algorithms.</p>
<a href='https://www.kaggle.com/uciml/mushroom-classification'>Mushroom Dataset</a>

<h2>Getting Started...</h2>
<p>Before getting started, you will need to install a couple things. First, make sure you have <a href='https://docs.continuum.io/anaconda/install'>Anaconda</a> installed on your computer. Then, use Anaconda's package manager to install 'pandas', 'numpy', 'matplotlib', 'graphviz', and 'sklearn'.</p>
<p>Also, download the mushroom dataset <a href='https://www.kaggle.com/uciml/mushroom-classification'>here.</a></p>
<p>After setting up the environment and dataset, create a jupyter notebook and import the following packages</p>


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from graphviz import Source
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn.grid_search import GridSearchCV
from operator import itemgetter
from os import system

<h3>Data Preparation</h3>
1.) Import the "mushrooms.csv" file and create a dataframe.

In [5]:
shrooms_df = pd.read_csv('mushrooms.csv')
shrooms_df.head(5)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


<p>2.) Check to make sure the data is clean. You can do this by going on <a href="https://www.kaggle.com/uciml/mushroom-classification">Kaggle's Website</a> to see the documentation for possible attribute values. Check to make sure all values are valid and look for any missing values</p>

In [6]:
poison = ['p', 'e']
cap_shape = ['b', 'c', 'x', 'f', 'k', 's']
cap_surface = ['f', 'g', 'y', 's']
cap_color = ['n', 'b', 'c', 'g', 'r', 'p', 'u', 'e', 'w', 'y']
bruises = ['t', 'f']
odor = ['a', 'l', 'c', 'y', 'f', 'm', 'n', 'p', 's']
gill_attachment = ['a', 'd', 'f', 'n']
gill_spacing = ['c', 'w', 'd']
gill_size = ['b', 'n']
gill_color = ['k', 'n', 'b', 'h', 'g', 'r', 'o', 'p', 'u', 'e', 'w', 'y']
stalk_shape = ['e', 't']
stalk_root = ['b', 'c', 'u', 'e', 'z', 'r']
stalk_surface_above_ring = ['f', 'y', 'k', 's']
stalk_surface_below_ring = ['f', 'y', 'k', 's']
stalk_color_above_ring = ['n', 'b', 'c', 'g', 'o', 'p', 'e', 'w', 'y']
stalk_color_below_ring = ['n', 'b', 'c', 'g', 'o', 'p', 'e', 'w', 'y']
veil_type = ['p', 'u']
veil_color = ['n', 'o', 'w', 'y']
ring_number = ['n', 'o', 't']
ring_type = ['c', 'e', 'f', 'l', 'n', 'p', 's', 'z']
spore_print_color = ['k', 'n', 'b', 'h', 'r', 'o', 'u', 'w', 'y']
population = ['a', 'c', 'n', 's', 'v', 'y']
habitat = ['g', 'l', 'm', 'p', 'u', 'w', 'd']

classifications = [poison, cap_shape, cap_surface, cap_color, bruises, \
                  odor, gill_attachment, gill_spacing, gill_size, \
                  gill_color, stalk_shape, stalk_root, stalk_surface_above_ring, \
                  stalk_surface_below_ring, stalk_color_above_ring, \
                  stalk_color_below_ring, veil_type, veil_color, \
                  ring_number, ring_type, spore_print_color, population, \
                  habitat]

printed = False

for index, row in shrooms_df.iterrows():
    for i in range(0, len(classifications)):
        if (not (row[i] in classifications[i])):
            shrooms_df.set_value(index, shrooms_df.columns.values[i], np.nan)

<p>3.) Split the dataset into training and test sets</p>

In [7]:
train_df = pd.concat([shrooms_df.iloc[0::3, :], shrooms_df.iloc[1::3, :]])
test_df = shrooms_df.iloc[2::3, :]

train_df.sort_index(inplace = True)
train_df.head(5)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m


<p>Unfortunatley, the decision trees implemented in scikit-learn only use numerical features (<a href="https://datascience.stackexchange.com/questions/5226/strings-as-features-in-decision-tree-random-forest">Link</a>), so we need to apply a one-hot encoding to our dataset since our values are not numbers.</p>
<p>4.) Apply a one hot encoding to the dataset</p>

In [8]:
encoded_train = pd.get_dummies(train_df, \
                               columns = list(train_df.columns.values)[1:])
encoded_train.head(5)

,class,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,p,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,1,0
1,e,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,p,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,e,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
6,e,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


Example decision tree 

In [9]:
y_train = encoded_train['class']
x_train = encoded_train[list(encoded_train.columns[1:])]

decision_tree = DecisionTreeClassifier(criterion = 'entropy', max_depth = 3) # min_samples_split=20, random_state=99)
decision_tree.fit(x_train, y_train)

dotfile = open("assets/dt.dot", 'w')
tree.export_graphviz(decision_tree, out_file = dotfile, feature_names=encoded_train.columns[1:])
dotfile.close()

#Note you must have graphviz installed to run this command
#If the return value is not 0, it did not execute correctly
system("dot -Tpng assets/dt.dot -o assets/dt.png")

0

<img src="assets/dt.png" height="600" width="600">

k-Fold Cross-Validation


In [10]:
def k_fold_cv (X, Y, classifier, param_grid, cv):
    grid_search = GridSearchCV(classifier, param_grid=param_grid, cv=cv)
    grid_search.fit(X,Y)
    return grid_search

def get_top_n_scores(grid_scores, n):
    top_n_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n]
    for i, score in enumerate(top_n_scores):
        print("Rank: " + str(i + 1))
        print("Mean Test score: " + str(score.mean_validation_score))
        print("Max-Depth: " + str(score.parameters) + "\n")
    
param_grid = {"max_depth": np.array(range(1, len(train_df.columns[1:])))}
dt_classifier = DecisionTreeClassifier()
grid_search = k_fold_cv(x_train, y_train, dt_classifier, param_grid, cv = 10)



get_top_n_scores(grid_search.grid_scores_, 5)


Rank: 1
Mean Test score: 0.9671344165435746
Max-Depth: {'max_depth': 8}

Rank: 2
Mean Test score: 0.9671344165435746
Max-Depth: {'max_depth': 10}

Rank: 3
Mean Test score: 0.9671344165435746
Max-Depth: {'max_depth': 11}

Rank: 4
Mean Test score: 0.9671344165435746
Max-Depth: {'max_depth': 13}

Rank: 5
Mean Test score: 0.9671344165435746
Max-Depth: {'max_depth': 14}



In [12]:
decision_tree = DecisionTreeClassifier(criterion = 'entropy', max_depth = 8) # min_samples_split=20, random_state=99)
decision_tree.fit(x_train, y_train)

dotfile = open("assets/optimal.dot", 'w')
tree.export_graphviz(decision_tree, out_file = dotfile, feature_names=encoded_train.columns[1:])
dotfile.close()

#Note you must have graphviz installed to run this command
#If the return value is not 0, it did not execute correctly
system("dot -Tpng assets/optimal.dot -o assets/optimal.png")

0

<img src="assets/optimal.png" height="600" width="600">